# Exercise 08: semantic similarity

First, let's create a function to calculate a *MinHash*:

In [ ]:
from datasketch import MinHash

def mh_digest (data):
    mh = MinHash(num_perm=512)

    for d in data:
        mh.update(d.encode('utf8'))

    return mh

Then we'll iterate through each parsed document, adding the keywords to the MinHash:

In [ ]:
import pynlp

files = ["a4.json", "a3.json", "a2.json", "a1.json"]

stopwords = pynlp.load_stopwords("stop.txt")
files_set = {}
files_mh = {}

for json_file in files:
    keywords = set([])

    for lex in pynlp.lex_iter(json_file):
        if (lex.pos != ".") and (lex.root not in stopwords):
            keywords.add(lex.root)

    files_set[json_file] = keywords
    files_mh[json_file] = mh_digest(keywords)

    print(json_file, keywords)

Let's compare the HTML documents, using a pairwise MinHash to approximate their Jaccard similarity:

In [ ]:
import itertools

sim = []

for i1, i2 in itertools.combinations(range(len(files)), 2):
    j = files_mh[files[i1]].jaccard(files_mh[files[i2]])
    sim.append((j, files[i1], files[i2],))

for jaccard, file1, file2 in sorted(sim, key=lambda x: x[0], reverse=True):
    print("%0.4f\t%s\t%s" % (jaccard, file1, file2))

Note the top-ranked ("most similar") pair, where both `html/article2.html` and `html/article3.html` are about machine learning. Take a look at their overlapping keywords:

In [ ]:
files_set["a3.json"] & files_set["a2.json"]